# 6.Softmax Classification

###### 2개 클래스를 분류할 때는 sigmoid함수가 유도된다
###### 3개 이상 클래스로 분류할 때는 softmax함수가 유도된다

P(class = i) = $\frac{e^i}{\sum_ie^i}$ 
모든 확률의 합=1
F.softmax(데이터, dim=0) // dim=0은 열별 합철수가 가위를 냈을 때 , softmax = [0.2, 0.3, 0.5]이면 $\\$
p(가위|철수=가위) = 0.2 $\\$
p(바위|철수=가위) = 0.3 $\\$
p(보|철수=가위) = 0.5 $\\$
이렇게 0과 1사이 확률값으로 표현할 수 있다

### Softmax함수 적용하기

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
z = torch.FloatTensor([1,2,3])
z

tensor([1., 2., 3.])

In [5]:
hypothesis = F.softmax(z, dim=0)
hypothesis

tensor([0.0900, 0.2447, 0.6652])

In [6]:
hypothesis.sum()

tensor(1.)

## 엔트로피
블로그 참조 : https://ratsgo.github.io/statistics/2017/09/22/information/ $\\$
x사건 : 동전이 앞면이 나오는 사건
0<=P(x)<=1
1. 정보량 I(x) = -logP(x) = -log0.5 // 정보량은 확률이 클 수록 줄이들고, 확률이 작을수록 커진다 => 확률이크다=거의 정해져있다=정보가없다 $\\$
$\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad$확률이작다=불확실하다=정보가 많다

2. 섀넌 엔트로피 H(x) = $E_{X-P}[I(x)] = E_{X-P}[-logP(x)]$ // 섀넌 엔트로피는 하나의 사건이 아니라 모든 사건의 정보량의 기대값 = I(동전앞면)+I(동전뒷정보량)의 기댓값
// 0.5*log0.5 + 0.5*log0.5=1 // p=0.5일때. 섀넌 엔트로피=1로 가장 크게 나타나고, P=0or1이면 섀년 엔트로피=0으로 가장 작게 나타남

3. KL Divergence(쿨백-라이블러 발산) KLD => $D_{kl}(P||Q) = E_{X-P}[log\frac{P(x)}{Q(x)}]$ // KLD는 두 확률분포의 차이를 계사하는 함수 P(x)는 실제값 & Q(x)는 예측값간 차이를 KLD를 이용해서 계산할 수 있지만, 비대칭으로 P와Q 위치가 바뀌면 KLD값도 바뀌므로 거리계산에서는 사용할 수 없음(절댓값, 제곱을 취하지 않으므로)

4. Cross entropy(크로스 엔트로피) H(P,Q) = E_{X-P}[-logQ(x)] = $-\sum_xP(x)logQ(x)$ // 두 확률분포가 교차로 곱해지는 엔트로피 계산방삭이다
P(x), Q(x)의 그래프를 예시로 두고 표현하면, KLD의 비대칭문제를 해결할 수 있다.

-KLD VS Cross entropy는 두 확률분포가 비슷하면 0에 가깝다. 즉 Q의 예측값을 점점 P에 가깝게 하므로써(실제값과 비슷하게) 학습과정을 진행하는데, 이때 두 엔트로피를 줄이는 방향으로 진행해 나가며 된다

$D_{kl}(P||Q) = -\sum_xlogP(x)\frac{Q(x)}{P(x)}$ = H(P.Q) - H(P) $\\$
H(P.Q) = H(P) + $D_{kl}(P||Q)$ 로 계산된다. H(P)는 실제 데이터분포이므로 학습과정에서 바뀌지 않는다. 즉 크로스 엔트로피를 줄인다는 것은 KLD를 줄인다는 것과 의미가 같다

### 크로스 엔트로피 적용하기

In [23]:
z = torch.rand(3,5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)

#sample=3, classes=5로 각 클래스에 속할 값을 알려줌
#해당 z값들은 범위의 제한이 없으므로 softmax함수를 적용해서 [0,1]사이 값으로 변형
z,hypothesis

(tensor([[0.0748, 0.9799, 0.5261, 0.8427, 0.6036],
         [0.6608, 0.8735, 0.9741, 0.1682, 0.5625],
         [0.8731, 0.8622, 0.8106, 0.1381, 0.1399]], requires_grad=True),
 tensor([[0.1124, 0.2779, 0.1765, 0.2423, 0.1908],
         [0.1952, 0.2415, 0.2671, 0.1193, 0.1769],
         [0.2572, 0.2544, 0.2416, 0.1233, 0.1235]], grad_fn=<SoftmaxBackward0>))

In [70]:
#실제 각 samples이 속하는 클래스(여기서는 랜덤지정)
y = torch.randint(5,(3,))
y

tensor([1, 3, 2])

In [71]:
y,y.unsqueeze(0), y.unsqueeze(1)

(tensor([1, 3, 2]),
 tensor([[1, 3, 2]]),
 tensor([[1],
         [3],
         [2]]))

In [72]:
#one-hot-encoding을 하기 위해 (3,5)행렬을 모두 0으로 초기화
y_one_hot = torch.zeros_like(hypothesis)

#scatter_ 밑줄을 쓰면 inplace=True반영
#dim=1 : 행을 기준으로 ont-hot-encoding를 할 것이다
#y.unsqieeze(1) => 각 행별 어느 인덱스에 값을 넣을지 지정해줌 => 1행에 1, 2행에2, 3행에 0번 인덱스에 값 반영
#src=1 : 실제 반영할 값
y_one_hot.scatter_(1,y.unsqueeze(1),1)

tensor([[0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])

In [74]:
#y_one_hot : 실제값(P(x)) = 5개의 클래스 중에서 하나만 1
#hypothesis : 예측값(Q(x)) = 5개의 클래스 중에서 하나만 1로 예측함
#크로스 엔트로피를 이용해서 P(x)*-logQ(x)계산
y_one_hot*-torch.log(hypothesis)

tensor([[0.0000, 1.2803, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 2.1262, 0.0000],
        [0.0000, 0.0000, 1.4205, 0.0000, 0.0000]], grad_fn=<MulBackward0>)

In [81]:
#각 sample별 합=>어차피 y_one_hot은 하나만 1이고 나머지는 0이므로
#1에 해당하는 P(x)*-logQ(x)값만 구함
(y_one_hot*-torch.log(hypothesis)).sum(dim=1)

tensor([1.2803, 2.1262, 1.4205], grad_fn=<SumBackward1>)

In [82]:
#우리가 구하고 싶은 크로스 엔트로피 값
(y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()

tensor(1.6090, grad_fn=<MeanBackward0>)

### torch.nn.functional를 이용해서 크로스 엔트로피 적용하기

위에서는 예측값z -> softmax로 적용시킴(hypothesis=Q(x)) $\\$
실제값(one-hot-encoding된 상태)P(x) 와 Q(x)를 이용해서 $-\sum_xP(x)logQ(x)$ 계산한다

In [86]:
#logQ(x)를 이렇게 계산해도 됨
torch.log(F.softmax(z, dim=1))

tensor([[-2.1855, -1.2803, -1.7342, -1.4175, -1.6567],
        [-1.6336, -1.4209, -1.3203, -2.1262, -1.7319],
        [-1.3580, -1.3689, -1.4205, -2.0930, -2.0912]], grad_fn=<LogBackward0>)

In [88]:
#logQ(x)를 F함수를 이용해서 계산
F.log_softmax(z,dim=1)

tensor([[-2.1855, -1.2803, -1.7342, -1.4175, -1.6567],
        [-1.6336, -1.4209, -1.3203, -2.1262, -1.7319],
        [-1.3580, -1.3689, -1.4205, -2.0930, -2.0912]],
       grad_fn=<LogSoftmaxBackward0>)

In [89]:
(y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()

tensor(1.6090, grad_fn=<MeanBackward0>)

In [91]:
#softmax를 적용시킨 Q(x) -> logQ(x)
(y_one_hot*-F.log_softmax(z,dim=1)).sum(dim=1).mean()

tensor(1.6090, grad_fn=<MeanBackward0>)

In [93]:
#logQ(x), P(x)를 인수로주면 크로스 엔트로피를 계산해주는 함수
F.nll_loss(F.log_softmax(z,dim=1), y)

tensor(1.6090, grad_fn=<NllLossBackward0>)

In [95]:
#예측값z, 정답값y를 인수로주면 크로스 엔트로피를 계산해주는 함수
F.cross_entropy(z,y)

tensor(1.6090, grad_fn=<NllLossBackward0>)

## 위에서 구한 softmax와 cross entropy를 이용해서 학습하는 방법

In [96]:
#8개의 sample데이터와 4개의 클래스가 존재함 + 각 클래스에 속할 점수값들이 있음
#우리는 4개의 클래스를 이용해서 3개의 분류를 할 예정임(0/1/2)

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
x_train, y_train

(tensor([[1., 2., 1., 1.],
         [2., 1., 3., 2.],
         [3., 1., 3., 4.],
         [4., 1., 5., 5.],
         [1., 7., 5., 5.],
         [1., 2., 5., 6.],
         [1., 6., 6., 6.],
         [1., 7., 7., 7.]]),
 tensor([2, 2, 2, 1, 1, 1, 0, 0]))

1. 예측값을 계산한다
2. 정답값label를 one-hot-encoding한다
3. 예측값-정답값을 이용해서 cost를 계산한다
4. gradient초기화-cost를이용한 gradient계산-계산한 gradient로 W,b업데이트

In [100]:
#각 클래스별 0/1/2의 가중치를 나타내므로 클래스=4, 분류=3으로 W를 표현함
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 1000

for epoch in range(nb_epochs+1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)

    #분류 실제값 y도 one-hot-encoding를 진행함
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

    cost = (y_one_hot * -torch.log(F.softmax(hypothesis,dim=1))).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


In [103]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 1000

for epoch in range(nb_epochs+1):
    z = x_train.matmul(W) + b

    #예측값 z를 softmax적용 + 정답값y를 one-hot-encoding 작업을 함수로 대체함
    cost = F.cross_entropy(z,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [104]:
#4개의 input -> 3개의 output으로 분류하는 모델
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)

    def forward(self,x):
        return self.linear(x)

In [106]:
model = SoftmaxClassifierModel()
model

SoftmaxClassifierModel(
  (linear): Linear(in_features=4, out_features=3, bias=True)
)

In [107]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(nb_epochs+1):
    prediction = model(x_train)

    cost = F.cross_entropy(prediction,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.528949
Epoch  100/1000 Cost: 0.643070
Epoch  200/1000 Cost: 0.558123
Epoch  300/1000 Cost: 0.503727
Epoch  400/1000 Cost: 0.459507
Epoch  500/1000 Cost: 0.420190
Epoch  600/1000 Cost: 0.383406
Epoch  700/1000 Cost: 0.347632
Epoch  800/1000 Cost: 0.311712
Epoch  900/1000 Cost: 0.275230
Epoch 1000/1000 Cost: 0.244543
